In [1]:
from PEPit import PEP
from PEPit.functions import SmoothStronglyConvexFunction
from PEPit.functions import ConvexFunction
from PEPit.primitive_steps import proximal_step
from math import sqrt

In [2]:
import numpy as np

In [3]:
# Problem setting

N = 4
L = 2

problem = PEP()

In [4]:
# Declare an L-smooth convex function 

f = problem.declare_function(
                SmoothStronglyConvexFunction,
                mu=0,   # Strong convexity param.=0
                L=L)    # Smoothness param.

# Declare an convex function 
h = problem.declare_function(ConvexFunction)

# Declare F=f+h
F = f + h

In [5]:
# Define its unique optimal point xs = x_* and its function value Fs = F(x_*)
xs = F.stationary_point()
Fs = F.value(xs)

# Define the starting point x0
x0 = problem.set_initial_point()

# Set the initial constraint that is the distance between x0 and x_*
problem.set_initial_condition( (x0 - xs) ** 2 <= 1 )

In [6]:
# Define the stepsizes

def get_theta(N):
    
    theta=np.zeros(N+1)
    
    theta[0]=1
    
    for i in range(N):
        
        theta[i+1]=(1 + sqrt(4 * theta[i] ** 2 + 1)) / 2
        
    theta[N]=(1 + sqrt(8 * theta[N-1] ** 2 + 1)) / 2
    
    return theta

def get_gamma(N):
    
    gamma=np.zeros(N)
    
    theta=get_theta(N)
    
    for i in range(N):
        gamma[i]=( 2*theta[i] / theta[N] ** 2 ) * ( theta[N] ** 2 - 2 * theta[i] ** 2 + theta[i] )
        
    return gamma

theta=get_theta(N)

gamma=get_gamma(N)

In [7]:
# Set initial points

y_new = x0

x = x0

z = x0

# Compute N steps of OptISTA from x0

for i in range(N):
    
    x_old = x
    
    z_old = z
    
    y_old = y_new
    
    
    
    y_new, _, _  = proximal_step( y_old - gamma[i] / L * f.gradient(x), h, gamma[i] / L)
    
    z = x_old + ( y_new - y_old ) / gamma[i]
    
    x = z + ( ( theta[i] - 1 ) /  theta[i+1]  ) * ( z - z_old )  + ( theta[i] / theta[i+1] ) * ( z - x_old )

In [8]:
# Set the performance measure

problem.set_performance_metric(F.value(y_new) -Fs)


In [9]:
# Solve the PEP
verbose = 1
pepit_verbose = max(verbose, 0)
OptISTA_rate = problem.solve(verbose=pepit_verbose)

(PEPit) Setting up the problem: size of the main PSD matrix: 13x13
(PEPit) Setting up the problem: performance measure is minimum of 1 element(s)
(PEPit) Setting up the problem: initial conditions and general constraints (1 constraint(s) added)
(PEPit) Setting up the problem: interpolation conditions for 2 function(s)
		 function 1 : 30 constraint(s) added
		 function 2 : 30 constraint(s) added
(PEPit) Setting up the problem: 0 lmi constraint(s) added
(PEPit) Compiling SDP
(PEPit) Calling SDP solver
(PEPit) Solver status: optimal (solver: MOSEK); optimal value: 0.05392723862045904
(PEPit) Postprocessing: solver's output is not entirely feasible (smallest eigenvalue of the Gram matrix is: -4.91e-09 < 0).
 Small deviation from 0 may simply be due to numerical error. Big ones should be deeply investigated.
 In any case, from now the provided values of parameters are based on the projection of the Gram matrix onto the cone of symmetric semi-definite matrix.


In [10]:
# Theoretical rate of OptISTA [Theorem 1]

theoretical_rate = L  / ( 2 * (theta[N] ** 2 - 1 ) ) 

In [12]:
# PEP guarantee equals Theoretical rate

print('*** worst-case performance of OptISTA ***')
print('\t PEP guarantee:\t f(y_N)-f_* <= {:.6} ||x0 - xs||^2'.format(OptISTA_rate ))
print('\t Theorem 1:\t f(y_N)-f_* <= {:.6} ||x0 - xs||^2 '.format(theoretical_rate))

*** worst-case performance of OptISTA ***
	 PEP guarantee:	 f(y_N)-f_* <= 0.0539272 ||x0 - xs||^2
	 Theorem 1:	 f(y_N)-f_* <= 0.0539272 ||x0 - xs||^2 
